In [2]:
import pandas as pd
import tensorflow as tf

from matplotlib import pyplot as plt
import ipywidgets as widgets
import cv2
import numpy as np
import os
import random
print("hello noobs")
print('axel was here')

hello noobs
axel was here


In [3]:
gestures = ["open_palm",
             "open_dorsal",
             "fist_palm",
             "fist_dorsal",
             "three_fingers_palm",
             "three_fingers_dorsal"]

img_path = 'img'
video_path = 'videos'
dataset = pd.read_csv('video_data.csv')
ids = ['102', '159', '294', '441', '564', '576', '609', '666', '711', '723']

columns=[]
#Lägger in alla "punkter" (kolumnerna i csv) i en lista, ex. palm_thumb_1_x, 
for col in dataset.columns[5:]:
    columns.append(col)

Metoden videosToFrames behöver köras endast en gång, då loopar den genom alla id's och sparar ner alla videos som frames i följande mappstruktur: 

/hand-classifier
    /img
        /open_palm
            /102
                0.jpg
                1.jpg osv...

Alla bilder tar mycket plats tillsammans dock, landar någonstans kring 3-4 GB så vet inte riktigt hur vi gör med det.

In [ ]:
def videosToFrames():
    n=0
    
    for current_id in ids:
        the_path = str(video_path + '/' + current_id)
        print('Current id: ' + str(current_id))
        for video_name in gestures:
            complete_video_path = str(the_path + '/' + video_name + '.webm')
            cap = cv2.VideoCapture(complete_video_path)
            i=0
            while(cap.isOpened()):
                ret, frame = cap.read()
                if ret == False:
                    break
                cv2.imwrite(img_path + '/' + video_name + '/' + current_id + '/' + str(i)+'.png', frame)
                i+=1
                
            cap.release()
            cv2.destroyAllWindows()
            print('video: ' + video_name + ' is done')
            
            
# Kör metoden
#videosToFrames()

In [2]:
def getTrainingImgs():
    limit=2

    training_data = []
    
    
    for row in dataset.itertuples(index=True, name='Frame'):
        current_id = str(row[1])
        gesture = str(row[4] + '_' + row[5])
        frame = row[3]
        
        path = 'img/'+gesture+'/'+current_id+'/{}.png'
        
        #print('current_id: ' + str(current_id))
        #print('current gesture: ' + gesture)
        #print('current frame: '+ str(frame))
        print('path: '+ path)
        
        location = path.format(frame)
        img = cv2.imread(location)
        
        #Loop through all points, i.e. the 40 columns
        for idx, column in enumerate(columns):
            
            #Make sure the loop doesnt run out of array
            if idx+1 < len(columns):
                    
                #Gets nextcolumn from next "string" in the columns array
                nextColumn=columns[idx+1]
                
            #Makes sure that each coordinate pair (x,y) is taken, basically jumps over each other column    
            if idx % limit == 0:
                label = column.replace('_x','')
                #Gets coordinates pairwise 
                coords = (int(getattr(row, column)), int(getattr(row, nextColumn)))
                    
                #checks that first coord in pair is not 0
                if(coords[0] > 1):
                    x1 = coords[0]-5
                    x2 = coords[0]+5
                    y1 = coords[1]-5
                    y2 = coords[1]+5
                                
                    training_img = img[y1:y2, x1:x2]
                    training_data.append([training_img, label])

getTrainingImgs()


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-b60e198a66f0>", line 47, in <module>
    getTrainingImgs()
  File "<ipython-input-2-b60e198a66f0>", line 7, in getTrainingImgs
    for row in dataset.itertuples(index=True, name='Frame'):
NameError: name 'dataset' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\User\Anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>

NameError: name 'dataset' is not defined

In [ ]:
random.shuffle(training_data)
for sample in training_data[:10]:
    print(sample[1])

In [ ]:
X_train = []
Y_train = []

for feature,label in training_data:
    X_train.append(feature)
    Y_train.append(label)

Y_train = np.array(Y)
X_train = np.array(X).reshape(-1,10,10,3)

encoder = sk.preprocessing.OneHotEncoder(dtype=np.float32)
encoder.fit(Y_train.reshape((-1, 1)))

Y_train_encoded = encoder.transform(Y_train.reshape((-1, 1))).toarray()
#Y_test_encoded = encoder.transform(Y_test.reshape((-1, 1))).toarray()

In [ ]:
def create_model():
    alpha=1e-4
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(11,11)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(400, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(10))
    model.add(tf.keras.layers.Softmax())

    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=alpha),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=[tf.keras.metrics.CategoricalAccuracy()]
                 )
  
    return model
                  
model = create_model()
model.summary()